# Tranfer learning VGG16 model

## Set up notebook

### Import dependencies

In [1]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [2]:
print(tf.__version__)

2.15.0


### Limit GPU usage

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Preparing data


### Import data from drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dir = Path("/content/drive/MyDrive/Newest_data_train")
test_dir = Path("/content/drive/MyDrive/Newest_data_test")
print("train data exist:", train_dir.exists())
print("test data exist:", test_dir.exists())

train data exist: True
test data exist: True


In [ ]:
train_img_count = len(list(train_dir.glob("*/*")))
test_img_count = len(list(test_dir.glob("*/*")))
print(train_img_count + test_img_count, "images found")

3147 images found


### Load data into training set and validation set

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
COLOR = "rgb"

train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=train_dir,
    shuffle=True,
    seed=716,
    image_size=IMAGE_SIZE,
    color_mode=COLOR,
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=test_dir,
    shuffle=True,
    seed=295,
    image_size=IMAGE_SIZE,
    color_mode=COLOR,
)
CLASS_NAMES = train_ds.class_names
NUM_CLASSES = len(CLASS_NAMES)
NUM_BATCH = int(train_ds.cardinality().numpy())

Found 2481 files belonging to 99 classes.
Found 657 files belonging to 99 classes.


In [ ]:
CLASS_NAMES

['Alex Lawther',
 'Amber Heart',
 'Angelia Jolie',
 'Avicii',
 'Ben Affleck',
 'Benzema',
 'Bill Gates',
 'Brie Larson',
 'Calvin Harris',
 'Chipu-error',
 'Chris Evans',
 'Chris Martin',
 'Chris Pratt',
 'Cillian Murphy',
 'Claudia Salas',
 'Cristiana Ronaldo',
 'Dakota Johnson',
 'Dat Ta',
 'David Beckham',
 'Doug McMillion',
 'Drake-error',
 'Duy Dat',
 'Elizabeth Olsen',
 'Elle Fanning',
 'Elon Musk',
 'Emilia Clarke',
 'Emily Blunt',
 'Gal Gadot',
 'Gordon Ramsey',
 'Hanni NewJeans',
 'Henry Cavil',
 'Ho Ngoc Ha',
 'Irene',
 'J.K.Rowling',
 'Jack Ma',
 'Jackie Chan',
 'Jason Momoa',
 'Jeff Bezos',
 'Jenna Ortega',
 'Jennie Blackpink',
 'Jennifer Lawrence',
 'Jessica Barden',
 'Jisoo-it',
 'Joe Biden',
 'Karen Gillan',
 'Keanu Reeves',
 'Kit Harington',
 'LeBron James-it',
 'Leonardo Dicaprio',
 'Lewis Hamilton-it',
 'Lily Collins',
 'Lisa-it',
 'Marc Marquez-it',
 'Marie Curie-it',
 'Mark Wahlberg',
 'Megan Fox',
 'Messi',
 'Micheal B Jordan',
 'Mina Twice',
 'Mohamed Salah',
 'Na

### Visualize the data


### Data augmentation


In [1]:
from keras import Sequential
from keras.layers import RandomBrightness, RandomFlip
from keras.layers import RandomRotation, RandomZoom

data_augmentation = Sequential(
    [
        RandomBrightness(0.3),
        RandomFlip("horizontal"),
        RandomRotation(0.2),
        RandomZoom(0.3),
    ],
    name="augmentation",
)

### Configure dataset for performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Training the model

### Import VGG16 model

In [ ]:
from keras.applications.vgg16 import VGG16
vgg16_model = VGG16(weights="imagenet",
                   include_top=False,
                   input_shape=IMAGE_SIZE + (3,),
                   )
vgg16_model.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### Transfer learning

In [ ]:
from keras import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Dense
from keras.layers import Rescaling, Flatten, Dropout, BatchNormalization

def create_model():
    model = Sequential(name="VGG16",
        layers = [
            Input(shape=IMAGE_SIZE + (3,), name="input"),
            # Preprocess
            data_augmentation,
            Rescaling(1.0 / 255),
            # Transfer learning
            vgg16_model,
            # Flatten and FC
            Flatten(),
            Dense(512, activation='relu'),
            BatchNormalization(),
            Dropout(0.6),
            Dense(512, activation = "relu"),
            BatchNormalization(),
            Dense(NUM_CLASSES, activation="softmax", name="output"),
        ]
    )

    model.compile(
        optimizer="adam",
        loss='sparse_categorical_crossentropy',
        metrics=["accuracy"],
    )

    return model


# Create a basic model instance
model = create_model()

### Build model

In [ ]:
model_path = "/content/drive/MyDrive/Model/vgg16.keras"
weights_path = "/content/drive/MyDrive/Model/vgg16_weights.keras"
model.build()
model.summary()

Model: "VGG16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 augmentation (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                             

### Train model

In [ ]:
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
EPOCHS = 200
FREQUENCY = 10

def custom_lr(epoch):
    if epoch < 50:
        return 0.001
    if epoch < 80:
      return 0.0005
    return 0.00001

lr = LearningRateScheduler(custom_lr)

mc = ModelCheckpoint(
    filepath=model_path,
    monitor="val_loss",
    verbose=1,
    save_weights_only=True,
    save_freq=FREQUENCY * NUM_BATCH,
)

es = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=40,
    start_from_epoch=EPOCHS,
)

history = model.fit(
    train_ds,
    verbose=1,
    epochs=EPOCHS,
    callbacks=[mc],
    validation_data=val_ds,
)
with open("/content/drive/MyDrive/Model/vgg16.txt", "w") as file:
    for p, l in history.history.items():
        file.write(str(p) + ": " + str(l) + "\n")

Epoch 1/200
78/78 [==============================] - 612s 7s/step - loss: 4.5918 - accuracy: 0.0645 - val_loss: 6.0285 - val_accuracy: 0.0441
Epoch 2/200
78/78 [==============================] - 14s 176ms/step - loss: 3.6554 - accuracy: 0.1640 - val_loss: 3.5616 - val_accuracy: 0.1887
Epoch 3/200
78/78 [==============================] - 16s 208ms/step - loss: 3.1512 - accuracy: 0.2213 - val_loss: 3.4750 - val_accuracy: 0.2496
Epoch 4/200
78/78 [==============================] - 14s 174ms/step - loss: 2.7557 - accuracy: 0.3128 - val_loss: 2.3086 - val_accuracy: 0.3881
Epoch 5/200
78/78 [==============================] - 13s 170ms/step - loss: 2.5164 - accuracy: 0.3482 - val_loss: 1.9936 - val_accuracy: 0.4505
Epoch 6/200
78/78 [==============================] - 13s 166ms/step - loss: 2.3684 - accuracy: 0.3716 - val_loss: 2.0316 - val_accuracy: 0.4505
Epoch 7/200
78/78 [==============================] - 13s 173ms/step - loss: 2.1806 - accuracy: 0.4079 - val_loss: 1.9672 - val_accuracy: 0

### Performance

In [ ]:
# create history loss and accuracy function
def plot_loss_acc(history):
    train_loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    epochs = range(len(train_loss))

    fig, ax = plt.subplots(1, 2)

    # Plot loss
    ax[1].set_title("Model loss")
    ax[1].set_xlabel("Epochs")
    ax[1].plot(epochs, train_loss, color="b", label="Train")
    ax[1].plot(epochs, val_loss, color="r", label="Validation")
    fig.legend()
    # Plot accuracy
    ax[0].set_title("Model accuracy")
    ax[0].set_xlabel("Epochs")
    ax[0].plot(epochs, acc, color="b", label="Train")
    ax[0].plot(epochs, val_acc, color="r", label="Validation")


    plt.show()

In [ ]:
plot_loss_acc(history)

### Save the model weights

In [ ]:
model.save_weights(weights_path)

## Prediction

### Load trained model

In [ ]:
trained_model = create_model()
trained_model.load_weights(weights_path)

### Show prediction

In [ ]:
plt.figure(figsize=(20, 20))

test_batch = val_ds.take(1)
prediction = trained_model.predict(test_batch)
for images, labels in test_batch:
    for i in range(25):
        img = images[i].numpy().astype("uint8")
        label = CLASS_NAMES[labels[i]]
        predict_label = CLASS_NAMES[np.argmax(prediction[i])]
        confidence = prediction[i][np.argmax(prediction[i])]

        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(img, cmap="gray")
        plt.title(f"Label: {label}\n Predict: {predict_label} {confidence :.2%}",fontsize = 12)
        plt.axis("off")

# plt.savefig("/content/drive/MyDrive/Model/result.png")

In [ ]:
test_img = keras.utils.load_img(
    "/content/drive/MyDrive/Dataset/Newest_data_test/vu duc dam/vu duc dam10.jpg",
    # "/content/drive/MyDrive/Model/410076200_1036268900763444_7352011443897460876_n.jpg",
    target_size=(224,224,3),
)
test_img

In [ ]:
pre = trained_model.predict(np.asarray(test_img)[None,:,:])
print(CLASS_NAMES[np.argmax(pre)], pre[0][np.argmax(pre)])

In [ ]:
# All test list
with open("/content/drive/MyDrive/Model/res.txt", "w") as file:
    for dir in test_dir.iterdir():
        print(dir)
        file.write(dir.name + "\n")
        for img_path in dir.iterdir():
            test_img = keras.utils.load_img(
                img_path,
                target_size=(224,224,3),
                color_mode="grayscale")
            test_prediction = trained_model.predict(np.asarray(test_img)[None,:,:])
            test_label = CLASS_NAMES[np.argmax(test_prediction)]
            test_confidence = test_prediction[0][np.argmax(test_prediction)]
            line = f"{img_path.name} Pre: {test_label} {test_confidence :.2%}"
            print(line)
            file.write("   "+ line + "\n")
